In [80]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 
rcParams['figure.figsize']=14,7
plt.style.use('seaborn-whitegrid')

In [70]:
bank =pd.read_csv('C:\\Vikalp_Private\\UIC\\Python\\assignments\\banking.csv',delimiter=';')
#converting all binary variables in 0 and 1
bank.y.replace(('yes','no'),(1,0),inplace=True)
bank.loan.replace(('yes','no'),(1,0),inplace=True)
bank.default.replace(('yes','no'),(1,0),inplace=True)
bank.housing.replace(('yes','no'),(1,0),inplace=True)
#bank.drop(['poutcome'], axis=1)


In [71]:
#checking distribution with single variables......to check for missing values................
bank['marital'].value_counts()

married     2797
single      1196
divorced     528
Name: marital, dtype: int64

In [74]:
bank['job'].value_counts()
#38 missing values found...........



management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: job, dtype: int64

In [6]:
bank['education'].value_counts()
#187 missing values found..................


secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64

In [7]:
bank['default'].value_counts()

0    4445
1      76
Name: default, dtype: int64

In [8]:
bank['housing'].value_counts()

1    2559
0    1962
Name: housing, dtype: int64

In [9]:
bank['loan'].value_counts()

0    3830
1     691
Name: loan, dtype: int64

In [12]:
bank['contact'].value_counts()
#replace missing values with mode of the coulumn.
bank['contact']=bank['contact'].replace('unknown','cellular')
#1324 missing values.................

cellular     4220
telephone     301
Name: contact, dtype: int64

In [11]:
bank['month'].value_counts()

may    1398
jul     706
aug     633
jun     531
nov     389
apr     293
feb     222
jan     148
oct      80
sep      52
mar      49
dec      20
Name: month, dtype: int64

In [13]:
bank['poutcome'].value_counts()
bank = bank.drop(['poutcome'], axis=1)
# 3705 missing values..................which is very large.....it is advisable to remove column.

In [14]:
bank['y'].value_counts()

0    4000
1     521
Name: y, dtype: int64

In [15]:
#changing category of each object columns..........................

bank.education = bank.education.astype('category')
bank.contact=bank.contact.astype('category')
bank.loan = bank.loan.astype('category')
bank.default=bank.default.astype('category')
bank.housing = bank.housing.astype('category')
bank.y = bank.y.astype('category')
bank.job = bank.job.astype('category')
bank.marital = bank.marital.astype('category')

In [18]:
#changing levels of categorical variables to numerics...........
contact = ['cellular', 'telephone']
ed = ['primary','secondary','tertiary']
marital = ['married','single','divorced']
bank.education = bank.education.astype("category",categories=ed).cat.codes
bank.contact = bank.contact.astype("category", categories= contact).cat.codes
bank.marital = bank.marital.astype("category", categories=marital).cat.codes


In [75]:
#checking any other missing value in Dataset
bank[bank.isnull().any(axis=1)]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y


In [76]:
#imputing missing values by building model between job and education
def cross_tab(df,f1,f2):
    jobs=list(df[f1].unique())
    edu=list(df[f2].unique())
    dataframes=[]
    for i in edu:
        dfi=df[df[f2]==i]
        dfijob=dfi.groupby(f1).count()[f2]
        dataframes.append(dfijob)
    xx=pd.concat(dataframes,axis=1)
    xx.columns=edu
    xx=xx.fillna(0)
    return xx
cross_tab(bank,'job','education')

,primary,secondary,tertiary,unknown
job,,,,
admin.,17,393,51,17
blue-collar,369,524,12,41
entrepreneur,26,58,73,11
housemaid,57,28,22,5
management,39,116,787,27
retired,80,105,31,14
self-employed,15,76,88,4
services,25,363,16,13
student,2,47,19,16


In [77]:
#using above cross tabulation method to impute missing value, we can determine that 
bank.loc[(bank['education']=='NaN') & (bank['job']=='management'), 'education'] = '2'
bank.loc[(bank['education']=='NaN') & (bank['job']=='services'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='housemaid'), 'education'] = '0'
bank.loc[(bank['education']=='NaN') & (bank['job']=='retired'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='self-employed'), 'education'] = '2'
bank.loc[(bank['education']=='NaN') & (bank['job']=='services'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='student'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='technician'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='unemployed'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='blue-collar'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='admin.'), 'education'] = '1'
bank.loc[(bank['education']=='NaN') & (bank['job']=='entrepreneur'), 'education'] = '2'
bank.loc[(bank['job'] == 'NaN') & (bank['education']=='0'), 'job'] = 'blue-collar'
bank.loc[(bank['job'] == 'NaN') & (bank['education']=='1'), 'job'] = 'technician'
bank.loc[(bank['job'] == 'NaN') & (bank['education']=='2'), 'job'] = 'management'
#all missing values imputed..................................perfectly.

In [23]:
#pdays contain -1 values which means that these customers were not contacted before.................
#we are simply transforming this column into different categories........
bank['Missing_pdays'] = 0
bank['less_5_pdays'] = 0
bank['greater_15_pdays'] = 0
bank['bet_5_15_pdays'] = 0
bank['Missing_pdays'][bank['pdays']==-1] = 1
bank['less_5_pdays'][bank['pdays']<5] = 1
bank['greater_15_pdays'][(bank['pdays']>15) & (bank['pdays']<999)] = 1
bank['bet_5_15_pdays'][(bank['pdays']>=5)&(bank['pdays']<=15)]= 1
bank_new = bank.drop('pdays', axis=1)
bank_with_dummies=pd.get_dummies(bank_new)# transforming dataset in dummy variables to make dataset fit for model testing......
bank_with_dummies

,age,marital,education,balance,contact,day,duration,campaign,previous,Missing_pdays,...,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,y_0,y_1
0,30,-1,-1,1787,-1,19,79,1,0,1,...,0,0,0,0,0,0,1,0,1,0
1,33,-1,-1,4789,-1,11,220,1,4,0,...,0,0,0,0,1,0,0,0,1,0
2,35,-1,-1,1350,-1,16,185,1,1,0,...,0,0,0,0,0,0,0,0,1,0
3,30,-1,-1,1476,-1,3,199,4,0,1,...,0,0,1,0,0,0,0,0,1,0
4,59,-1,-1,0,-1,5,226,1,0,1,...,0,0,0,0,1,0,0,0,1,0
5,35,-1,-1,747,-1,23,141,2,3,0,...,0,0,0,0,0,0,0,0,1,0
6,36,-1,-1,307,-1,14,341,1,2,0,...,0,0,0,0,1,0,0,0,1,0
7,39,-1,-1,147,-1,6,151,2,0,1,...,0,0,0,0,1,0,0,0,1,0
8,41,-1,-1,221,-1,14,57,2,0,1,...,0,0,0,0,1,0,0,0,1,0
9,43,-1,-1,-88,-1,17,313,1,2,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
bank_with_dummies.dtypes

age                  int64
marital               int8
education            int64
balance              int64
contact               int8
day                  int64
duration             int64
campaign             int64
previous             int64
Missing_pdays        int64
less_5_pdays         int64
greater_15_pdays     int64
bet_5_15_pdays       int64
job_admin.           uint8
job_blue-collar      uint8
job_entrepreneur     uint8
job_housemaid        uint8
job_management       uint8
job_retired          uint8
job_self-employed    uint8
job_services         uint8
job_student          uint8
job_technician       uint8
job_unemployed       uint8
job_unknown          uint8
default_0            uint8
default_1            uint8
housing_0            uint8
housing_1            uint8
loan_0               uint8
loan_1               uint8
month_apr            uint8
month_aug            uint8
month_dec            uint8
month_feb            uint8
month_jan            uint8
month_jul            uint8
m

In [29]:
#function for feature removal. I am removing fetures which are not neccessary now or will be redundant in model fitting....

def featureselection(df,f):
    #removes features
    df=df.drop(f,axis=1)
    return df


In [32]:
#dropping features which are not used.......
features_removed = ['Missing_pdays','job_unknown','default_0','y_0','housing_0','loan_0']
bank_final = featureselection(bank_with_dummies, features_removed)

In [35]:
bank_final.dtypes

age                  int64
marital               int8
education            int64
balance              int64
contact               int8
day                  int64
duration             int64
campaign             int64
previous             int64
less_5_pdays         int64
greater_15_pdays     int64
bet_5_15_pdays       int64
job_admin.           uint8
job_blue-collar      uint8
job_entrepreneur     uint8
job_housemaid        uint8
job_management       uint8
job_retired          uint8
job_self-employed    uint8
job_services         uint8
job_student          uint8
job_technician       uint8
job_unemployed       uint8
default_1            uint8
housing_1            uint8
loan_1               uint8
month_apr            uint8
month_aug            uint8
month_dec            uint8
month_feb            uint8
month_jan            uint8
month_jul            uint8
month_jun            uint8
month_mar            uint8
month_may            uint8
month_nov            uint8
month_oct            uint8
m

In [38]:
#training and test data partition........................

from sklearn.model_selection import train_test_split

x_bank_final = bank_final[bank_final.columns[1:38]]
y_bank_final = bank_final[bank_final.columns[38]]

X_train, X_test, y_train, y_test = train_test_split(x_bank_final,y_bank_final,test_size =.10)

In [39]:
#running and checking parameteres on test data set..................................
SEED = 1234
models = []

models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('SVC', SVC()))


result = []
model_names = []
scoring = ['accuracy', 'recall', 'precision', 'roc_auc']

for model_name, model in models:
    kfold = KFold(n_splits=10, random_state=SEED)
    cv_results1 = cross_val_score(model, X_train, y_train, cv = kfold, scoring=scoring[0])
    cv_results2 = cross_val_score(model, X_train, y_train, cv = kfold, scoring=scoring[1])
    cv_results3 = cross_val_score(model, X_train, y_train, cv = kfold, scoring=scoring[2])
    cv_results4 = cross_val_score(model, X_train, y_train, cv = kfold, scoring=scoring[3])
    model_names.append(model_name)
    msg = "%s:\n ACCURACY = %f, RECALL=(%f), PRECISION=(%f), ROC-AUC=(%f)" % (model_name, cv_results1.mean(),cv_results2.mean(), cv_results3.mean(), cv_results4.mean())
    print(msg)

LR:
 ACCURACY = 0.891592, RECALL=(0.226977), PRECISION=(0.566672), ROC-AUC=(0.881424)
KNN:
 ACCURACY = 0.873155, RECALL=(0.167228), PRECISION=(0.376967), ROC-AUC=(0.693902)
DT:
 ACCURACY = 0.865047, RECALL=(0.401500), PRECISION=(0.392141), ROC-AUC=(0.649715)
SVC:
 ACCURACY = 0.884954, RECALL=(0.000000), PRECISION=(0.000000), ROC-AUC=(0.685587)


In [64]:
#confusion matrix and cross validation score for logit model....................
#logistics regression model is the best model for this case.......
logistic = LogisticRegression(C=1e5)
logistic.fit(X_train,y_train).score(X_test,y_test)
print(logistic.coef_) 
logpred=logistic.predict(X_test) 
print(logpred)
confusion_matrix(y_test,logpred)#performance on validation set
kfold = KFold(n_splits=10, random_state=SEED)
cv_results1 = cross_val_score(logistic, X_test, y_test, cv = kfold)#calculate cross validation score............
cv_results1.mean()

[[ 5.35402980e-01  5.35402980e-01  8.95935695e-06  5.35402980e-01
   5.42996287e-03  4.16151991e-03 -7.16384199e-02 -4.41400442e-03
  -7.27845731e-01  3.13497031e-01 -1.21054281e-01  1.88472583e-01
  -4.75448911e-01 -2.64618007e-01 -5.29061720e-01  2.45329326e-01
   5.61328118e-01 -1.14738668e-01  2.55157787e-02  4.70657889e-01
  -1.08258999e-01 -5.13528485e-01  3.34033045e-01 -4.79827661e-01
  -5.93604606e-01 -8.38911185e-02 -2.73618547e-01  2.27979999e-01
   4.90010516e-02 -9.64628123e-01 -6.61170209e-01 -4.23280076e-01
   1.60008282e+00 -1.12737314e+00 -8.92900835e-01  1.29528041e+00
   7.19114789e-01]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 

0.9005314009661836

In [65]:
#confusion matrix and cross validation score for KNN model....................
KNN = KNeighborsClassifier()
KNN.fit(X_train,y_train).score(X_test,y_test)

KNNpred=KNN.predict(X_test) 
print(KNNpred)
confusion_matrix(y_test,KNNpred)#performance on validation set

cv_results2 = cross_val_score(KNN, X_test, y_test, cv = kfold)#calculate cross validation score............
cv_results2.mean()

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0]


0.8827053140096618

In [66]:
#confusion matrix and cross validation score for DT model....................
DT = DecisionTreeClassifier()
DT.fit(X_train,y_train).score(X_test,y_test)

DTpred=DT.predict(X_test) 
print(DTpred)
confusion_matrix(y_test,DTpred)#performance on validation set
cv_results3 = cross_val_score(DT, X_test, y_test, cv = kfold)#calculate cross validation score............
cv_results3.mean()

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]


0.841062801932367

In [67]:
#confusion matrix and cross validation score for svc model....................
svc =SVC()
svc.fit(X_train,y_train).score(X_test,y_test)

svcpred=svc.predict(X_test) 
print(svcpred)
confusion_matrix(y_test,svcpred)#performance on validation set
cv_results4 = cross_val_score(svc, X_test, y_test, cv = kfold)#calculate cross validation score............
cv_results4.mean()

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


0.8828019323671497

In [ ]:
#we cannot run linear regression as target class is not continuous...it is a binary class.